In [1]:
import pickle
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

print("데이터 로딩 중...")
file_path = "mon_standard.pkl"
with open(file_path, 'rb') as fi:
    data = pickle.load(fi)


데이터 로딩 중...


In [4]:
# 데이터 전처리
X_time = []
y = []
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE

    for sample in data[i]:
        time_seq = []
        cumulative_size = 0
        for index, c in enumerate(sample):
            time_seq.append(abs(c))
        X_time.append(time_seq)
        y.append(label)

# 각 배열의 길이를 최대 길이에 맞춰 늘리고 부족한 부분은 0으로 채웁니다.
max_length = max(len(arr) for arr in X_time)

def pad_sequence(sequence, max_length):
    return sequence + [0] * (max_length - len(sequence))

X_time_padded = [pad_sequence(arr, max_length) for arr in X_time]

X_time_array = np.array(X_time_padded)
X_train_time, X_test_time, y_train_time, y_test_time = train_test_split(X_time_array, y, test_size=0.2, random_state=42)

# StandardScaler를 사용하여 X_time 데이터 정규화
scaler = StandardScaler()
X_train_time_scaled = scaler.fit_transform(X_train_time)
X_test_time_scaled = scaler.transform(X_test_time)

In [3]:
# svm
svm_model_time = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_time.fit(X_train_time_scaled, y_train_time)

y_pred_svm_time = svm_model_time.predict(X_test_time_scaled)
accuracy_svm_time = accuracy_score(y_test_time, y_pred_svm_time)

print(f"svm 모델 정확도 (X_time): {accuracy_svm_time * 100:.2f}%")

svm 모델 정확도 (X_time): 50.21%


In [4]:
# rf
rf_model_time = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_time.fit(X_train_time_scaled, y_train_time)

y_pred_rf_time = rf_model_time.predict(X_test_time_scaled)
accuracy_rf_time = accuracy_score(y_test_time, y_pred_rf_time)
print(f"랜덤 포레스트 모델 정확도 (time): {accuracy_rf_time * 100:.2f}%")

랜덤 포레스트 모델 정확도 (time): 45.61%


In [5]:
# knn
knn_model_time = KNeighborsClassifier(n_neighbors=5)
knn_model_time.fit(X_train_time_scaled, y_train_time)

y_pred_knn_time = knn_model_time.predict(X_test_time_scaled)

accuracy_knn_time = accuracy_score(y_test_time, y_pred_knn_time)
print(f"KNN 모델 정확도 (X_time): {accuracy_knn_time * 100:.2f}%")

KNN 모델 정확도 (X_time): 32.32%
